In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# === 1. Load and Prepare the Data ===
df = pd.read_csv("alzheimers_disease_data.csv")  # Replace with your file path
df = df.drop(columns=["PatientID", "DoctorInCharge"])

# Split features and target
X = df.drop(columns=["Diagnosis"]).values
y = df["Diagnosis"].values

# Normalize features using PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32)
X_mean = X_tensor.mean(dim=0)
X_std = X_tensor.std(dim=0)
X_tensor = (X_tensor - X_mean) / X_std

y_tensor = torch.tensor(y, dtype=torch.float32)

# Train/test split manually (80/20)
num_samples = X_tensor.shape[0]
split_index = int(num_samples * 0.8)
X_train, X_test = X_tensor[:split_index], X_tensor[split_index:]
y_train, y_test = y_tensor[:split_index], y_tensor[split_index:]

# === 2. Define the Neural Network ===
class SimpleFFNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Sigmoid()
    )

    def forward(self, x):
        return self.net(x)

model = SimpleFFNN(input_dim=X_train.shape[1])

# === 3. Define Loss and Optimizer ===
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# === 4. Training Loop ===
epochs = 300
for epoch in range(epochs):
    model.train()
    output = model(X_train).squeeze()
    loss = criterion(output, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 30 == 0:
        print(f"Epoch {epoch} - Loss: {loss.item():.4f}")

# === 5. Evaluation ===
model.eval()
with torch.no_grad():
    y_pred = model(X_test).squeeze()
    y_pred_labels = (y_pred > 0.5).float()
    accuracy = (y_pred_labels == y_test).float().mean().item()
    print(f"\n✅ Test Accuracy: {accuracy:.4f}")

Epoch 0 - Loss: 0.7174
Epoch 30 - Loss: 0.1197
Epoch 60 - Loss: 0.0062
Epoch 90 - Loss: 0.0005
Epoch 120 - Loss: 0.0002
Epoch 150 - Loss: 0.0002
Epoch 180 - Loss: 0.0001
Epoch 210 - Loss: 0.0001
Epoch 240 - Loss: 0.0001
Epoch 270 - Loss: 0.0001

✅ Test Accuracy: 0.7395
